In [ ]:
import random
random.seed(0)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from lifelines import CoxPHFitter
from lifelines.utils import restricted_mean_survival_time
from lifelines.statistics import proportional_hazard_test

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
original_data = pd.read_csv('Book1.csv')
original_data = original_data.drop(['p1','p2','p3','p4','p5','s1','s2','s3','s4','s5','c1','c2','c3','c4','c5','e1','e2','e3','e4','e5'], axis=1)

scores = pd.read_csv('factors.csv')
original_data = original_data.join(scores)
original_data.head(2)

In [ ]:
education = ['No formal education', 'Elementary', 'High School', 'College', 'Masters', 'PHd']
income = ['Less than  9,520', ' 9,520 to  21,194', ' 21,194 -  43,828', ' 43,828 -  76,669', ' 76,669 -  131,484', ' 131,484 -  219,140', ' 219,140 and up']
gender = ['Female', 'Male']

oe = OrdinalEncoder(categories=[education, education, income, gender])
le = LabelEncoder()

event = pd.DataFrame(le.fit_transform(original_data['event']), columns=['event'])
education_and_income = pd.DataFrame(oe.fit_transform(original_data[['f-edu', 'm-edu', 'income', 'gender']]), columns=['F-edu', 'M-edu', 'Income', 'Gender'])

In [ ]:
df = original_data.drop(['gender', 'f-edu', 'm-edu', 'income', 'event'], axis=1)
df = df.join(event)
df = df.join(education_and_income)
df.head()

In [ ]:
model = CoxPHFitter()
model.fit(df, 'duration', 'event')
model.print_summary(model="Cox regression", decimals=3)

In [ ]:
results = model.log_likelihood_ratio_test()
results

In [ ]:
survival_function = model.baseline_survival_
survival_function.to_csv('survival function.csv')
survival_function

In [ ]:
rmst = restricted_mean_survival_time(model.baseline_survival_)

sns.lineplot(survival_function.index, survival_function['baseline survival'], drawstyle="steps-post")
plt.title('Survival function')
plt.text(20, .98, f'Mean survival time: {round(rmst, 2)}')
plt.xlabel('Duration (weeks)')
plt.show()

In [ ]:
baseline_cumhaz = model.baseline_cumulative_hazard_
baseline_cumhaz.to_csv('baseline cumhazard.csv')
baseline_cumhaz

In [ ]:
sns.lineplot(baseline_cumhaz.index, baseline_cumhaz['baseline cumulative hazard'], drawstyle="steps-post")
plt.title('Cumulative hazard')
plt.xlabel('Duration (weeks)')

In [ ]:
hazard_rate = model.baseline_hazard_ 
hazard_rate.to_csv('hazard rate.csv')
hazard_rate

In [ ]:
sns.lineplot(hazard_rate.index, hazard_rate['baseline hazard'])
plt.title('Hazard rate')
plt.xlabel('Duration (weeks)')

In [ ]:
model.plot_partial_effects_on_outcome('Gender', [0, 1])
plt.xlabel('Duration (weeks)')

In [ ]:
model.plot_covariate_groups('Economic factor', values=[-1, -0.5, .5, 1])
plt.xlabel('Duration (weeks)')


In [ ]:
model.plot_covariate_groups('Social factor', values=[-1, -0.5, .5, 1])
plt.xlabel('Duration (weeks)')


In [ ]:
model.plot_covariate_groups('Gender', [0, 1])

In [ ]:
res = model.check_assumptions(
    df, 
    p_value_threshold=0.05, 
    show_plots=True, 
    advice=True
)

res

In [ ]:
results = proportional_hazard_test(model, df, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

In [ ]:
from bioinfokit.analys import stat

ctab = pd.crosstab(df.Gender, df.event)

res = stat()

res.chisq(df=ctab)

chi_square_test_res = pd.DataFrame(
    [
        [1,18.298,'<0.001'],
        [1,18.119,'<0.001']
    ],
    columns=['df', 'Chi-square', 'p'],
    index = ['Pearson','Log-likelihood']
)

chi_square_test_res.to_csv('Chi square result.csv')
chi_square_test_res
